In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import kagglehub

/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
mbti_personality_type_twitter_dataset_path = kagglehub.dataset_download('mazlumi/mbti-personality-type-twitter-dataset')

In [16]:
glove6b300dtxt_path = kagglehub.dataset_download('thanakomsn/glove6b300dtxt')

In [4]:
print(mbti_personality_type_twitter_dataset_path)
print(glove6b300dtxt_path)

/home/guilherme/.cache/kagglehub/datasets/mazlumi/mbti-personality-type-twitter-dataset/versions/1
/home/guilherme/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [5]:
import os

for dirname, _, filenames in os.walk(mbti_personality_type_twitter_dataset_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

for dirname, _, filenames in os.walk(glove6b300dtxt_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/guilherme/.cache/kagglehub/datasets/mazlumi/mbti-personality-type-twitter-dataset/versions/1/twitter_MBTI.csv
/home/guilherme/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1/glove.6B.300d.txt


In [2]:
# df = pd.read_csv(mbti_personality_type_twitter_dataset_path+'/twitter_MBTI.csv')
df = pd.read_pickle('cleaned_dataset.pickle')
df.head()

,Unnamed: 0,text,label,cleaned_text,cleaned_text_separated,tfidf_embedding,glove_embedding
0,0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj,pope infallible catholic dogma doesnt mean per...,"[pope infallible catholic dogma doesnt mean, p...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019...","[-0.04726143, -0.060535315, -0.057487372, 0.00..."
1,1,@Hispanthicckk Being you makes you look cute||...,intj,makes look cute fun peeling bored less sweetie...,"[makes look cute, fun peeling, bored, less, sw...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.049909562, -0.022831392, -0.028013704, -0...."
2,2,@Alshymi Les balles sont réelles et sont tirée...,intj,les balles sont relles et sont tires trs rapid...,[les balles sont relles et sont tires trs rapi...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022...","[-0.02637649, 0.002697029, -0.050253987, -0.02..."
3,3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj,im like entp idiotic hey boy want watch twitch...,"[im like entp idiotic, hey boy want watch twit...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0019069636, -0.0014414503, 0.00036121352, ..."
4,4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj,give pica since childhood say qubool hai dm ge...,"[give pica since childhood, say qubool hai dm,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0427088751511...","[-0.08774337, -0.06754079, -0.03461329, 0.0125..."


## Data cleaning

In [19]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re

# Download NLTK data files (if not already done)
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

# Define stopwords (optional)
stop_words = set(stopwords.words('english'))

# Function to clean a single tweet
def clean_tweet(tweet):
    # Remove HTML tags
    tweet = re.sub(r'<.*?>', '', tweet)
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+', '', tweet)
    # Remove mentions
    tweet = re.sub(r'@\w+', '', tweet)
    # Remove special characters and digits
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)
    # Convert to lowercase
    tweet = tweet.lower()
    # Tokenize and remove stopwords (optional)
    tokens = word_tokenize(tweet)
    cleaned_tweet = ' '.join([word for word in tokens if word not in stop_words])
    return cleaned_tweet

# Split tweets by '|||' and clean them
def process_user_tweets(tweets):
    # Split tweets
    tweet_list = tweets.split('|||')
    # Clean each tweet
    cleaned_tweets = [clean_tweet(tweet) for tweet in tweet_list]
    # Combine cleaned tweets
    return ' '.join(cleaned_tweets)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guilherme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/guilherme/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/guilherme/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [31]:
# Apply cleaning to the dataset
df['cleaned_text'] = df['text'].apply(process_user_tweets)
df.head()

,Unnamed: 0,text,label,cleaned_text
0,0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj,pope infallible catholic dogma doesnt mean per...
1,1,@Hispanthicckk Being you makes you look cute||...,intj,makes look cute fun peeling bored less sweetie...
2,2,@Alshymi Les balles sont réelles et sont tirée...,intj,les balles sont relles et sont tires trs rapid...
3,3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj,im like entp idiotic hey boy want watch twitch...
4,4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj,give pica since childhood say qubool hai dm ge...


In [32]:
df['cleaned_text_separated'] = df['text'].map(lambda x: x.split('|||')).\
                                apply(lambda x: [clean_tweet(tweet) for tweet in x])
df.head()

,Unnamed: 0,text,label,cleaned_text,cleaned_text_separated
0,0,@Pericles216 @HierBeforeTheAC @Sachinettiyil T...,intj,pope infallible catholic dogma doesnt mean per...,"[pope infallible catholic dogma doesnt mean, p..."
1,1,@Hispanthicckk Being you makes you look cute||...,intj,makes look cute fun peeling bored less sweetie...,"[makes look cute, fun peeling, bored, less, sw..."
2,2,@Alshymi Les balles sont réelles et sont tirée...,intj,les balles sont relles et sont tires trs rapid...,[les balles sont relles et sont tires trs rapi...
3,3,"I'm like entp but idiotic|||Hey boy, do you wa...",intj,im like entp idiotic hey boy want watch twitch...,"[im like entp idiotic, hey boy want watch twit..."
4,4,@kaeshurr1 Give it to @ZargarShanif ... He has...,intj,give pica since childhood say qubool hai dm ge...,"[give pica since childhood, say qubool hai dm,..."


### Tokenization

In [7]:
from nltk.tokenize import word_tokenize

# Tokenize aggregated tweets
df['tokens_aggregated_nltk'] = df['cleaned_text'].apply(word_tokenize)

# Tokenize separated tweets
df['tokens_separated_nltk'] = df['cleaned_text_separated'].apply(
    lambda tweet_list: [word_tokenize(tweet) for tweet in tweet_list]
)

print(df[['tokens_aggregated_nltk', 'tokens_separated_nltk']].head())

                              tokens_aggregated_nltk  \
0  [pope, infallible, catholic, dogma, doesnt, me...   
1  [makes, look, cute, fun, peeling, bored, less,...   
2  [les, balles, sont, relles, et, sont, tires, t...   
3  [im, like, entp, idiotic, hey, boy, want, watc...   
4  [give, pica, since, childhood, say, qubool, ha...   

                               tokens_separated_nltk  
0  [[pope, infallible, catholic, dogma, doesnt, m...  
1  [[makes, look, cute], [fun, peeling], [bored],...  
2  [[les, balles, sont, relles, et, sont, tires, ...  
3  [[im, like, entp, idiotic], [hey, boy, want, w...  
4  [[give, pica, since, childhood], [say, qubool,...  


In [8]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer (e.g., BERT tokenizer)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize aggregated tweets
df['tokens_aggregated_bert'] = df['cleaned_text'].apply(
    lambda text: tokenizer.encode(text, max_length=512, truncation=True)
)

# Tokenize separated tweets without truncation (each tweet is likely shorter)
df['tokens_separated_bert'] = df['cleaned_text_separated'].apply(
    lambda tweet_list: [tokenizer.encode(tweet, max_length=512, truncation=True) for tweet in tweet_list]
)

print(df[['tokens_aggregated_bert', 'tokens_separated_bert']].head())


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

                              tokens_aggregated_bert  \
0  [101, 4831, 1999, 13976, 7028, 3234, 3899, 286...   
1  [101, 3084, 2298, 10140, 4569, 28241, 11471, 2...   
2  [101, 4649, 3608, 2229, 2365, 2102, 2128, 2043...   
3  [101, 10047, 2066, 4372, 25856, 10041, 2594, 4...   
4  [101, 2507, 27263, 2050, 2144, 5593, 2360, 242...   

                               tokens_separated_bert  
0  [[101, 4831, 1999, 13976, 7028, 3234, 3899, 28...  
1  [[101, 3084, 2298, 10140, 102], [101, 4569, 28...  
2  [[101, 4649, 3608, 2229, 2365, 2102, 2128, 204...  
3  [[101, 10047, 2066, 4372, 25856, 10041, 2594, ...  
4  [[101, 2507, 27263, 2050, 2144, 5593, 102], [1...  


### Embeddings

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use aggregated tweets for TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
tfidf_embeddings = vectorizer.fit_transform(df['cleaned_text'])

# Save embeddings to DataFrame
df['tfidf_embedding'] = list(tfidf_embeddings.toarray())

print(df[['cleaned_text', 'tfidf_embedding']].head())

                                        cleaned_text  \
0  pope infallible catholic dogma doesnt mean per...   
1  makes look cute fun peeling bored less sweetie...   
2  les balles sont relles et sont tires trs rapid...   
3  im like entp idiotic hey boy want watch twitch...   
4  give pica since childhood say qubool hai dm ge...   

                                     tfidf_embedding  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.019...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0427088751511...  


In [34]:
import numpy as np
from gensim.models import KeyedVectors

# Load pre-trained GloVe vectors (download from https://nlp.stanford.edu/projects/glove/)
glove_path = glove6b300dtxt_path + '/glove.6B.300d.txt'  # Adjust path and file size as needed
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, no_header=True)

# Function to get sentence embeddings by averaging word embeddings
def get_sentence_embedding(text, word_vectors):
    tokens = text.split()  # Simple split, adapt if needed
    embeddings = [word_vectors[word] for word in tokens if word in word_vectors]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)

# Apply to aggregated tweets
df['glove_embedding'] = df['cleaned_text'].apply(
    lambda text: get_sentence_embedding(text, word_vectors)
)

print(df[['cleaned_text', 'glove_embedding']].head())


                                        cleaned_text  \
0  pope infallible catholic dogma doesnt mean per...   
1  makes look cute fun peeling bored less sweetie...   
2  les balles sont relles et sont tires trs rapid...   
3  im like entp idiotic hey boy want watch twitch...   
4  give pica since childhood say qubool hai dm ge...   

                                     glove_embedding  
0  [-0.04726143, -0.060535315, -0.057487372, 0.00...  
1  [-0.049909562, -0.022831392, -0.028013704, -0....  
2  [-0.02637649, 0.002697029, -0.050253987, -0.02...  
3  [-0.0019069636, -0.0014414503, 0.00036121352, ...  
4  [-0.08774337, -0.06754079, -0.03461329, 0.0125...  


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch

# # Load a pre-trained BERT model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Function to get BERT embeddings for aggregated tweets
# def get_bert_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
#     with torch.no_grad():
#         outputs = model(**inputs)
#     # Take the CLS token's embedding for classification
#     cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
#     return cls_embedding.numpy()

# # Apply to aggregated tweets
# df['bert_embedding'] = df['cleaned_text'].apply(get_bert_embedding)

# print(df[['cleaned_text', 'bert_embedding']].head())


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [35]:
# Save the DataFrame to a CSV file
output_path = 'cleaned_dataset.pickle'
df.to_pickle(output_path)

print(f"DataFrame saved to {output_path}")

DataFrame saved to cleaned_dataset.pickle


### One Hot Encoding

In [3]:
from sklearn.preprocessing import OneHotEncoder

# Assuming 'label' column exists in the dataframe
encoder = OneHotEncoder(sparse_output=False)
labels_one_hot = encoder.fit_transform(df[['label']])

# Add to the dataframe or save separately
df['label_one_hot'] = list(labels_one_hot)

## Pipeline

#### Addressing data imbalance

In [15]:
# Check the distribution of classes
print(df['label'].value_counts())

label
infp    1282
infj    1057
intp     811
intj     781
enfp     729
entp     577
enfj     518
isfp     367
isfj     364
istp     327
entj     279
istj     259
esfp     174
esfj     105
estp     100
estj      81
Name: count, dtype: int64


In [23]:
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import Pipeline as ImbPipeline

# # Choose between SMOTE or RandomUnderSampler
# oversample = SMOTE(random_state=42)  # Use for oversampling
# undersample = RandomUnderSampler(random_state=42)  # Use for undersampling

# # Example: Oversample
# X = df['tfidf_embedding']  # Use your tfidf embedding matrix
# y = df['label']  # Original labels
# X_resampled, y_resampled = oversample.fit_resample(X, y)

# # Encode labels as one-hot
# y_resampled_one_hot = encoder.fit_transform(y_resampled.values.reshape(-1, 1))


In [4]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Convert tfidf_embedding to NumPy array
X = np.array(df['tfidf_embedding'].tolist())

# Ensure labels are categorical
y = df['label']

# Apply SMOTE for oversampling
oversample = SMOTE(random_state=42)
X_resampled, y_resampled = oversample.fit_resample(X, y)

# Encode labels as one-hot
y_resampled_one_hot = encoder.fit_transform(y_resampled.values.reshape(-1, 1))

print("Resampled X shape:", X_resampled.shape)
print("Resampled y distribution:", np.unique(y_resampled, return_counts=True))


/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Resampled X shape: (20512, 5000)
Resampled y distribution: (array(['enfj', 'enfp', 'entj', 'entp', 'esfj', 'esfp', 'estj', 'estp',
       'infj', 'infp', 'intj', 'intp', 'isfj', 'isfp', 'istj', 'istp'],
      dtype=object), array([1282, 1282, 1282, 1282, 1282, 1282, 1282, 1282, 1282, 1282, 1282,
       1282, 1282, 1282, 1282, 1282]))


#### Classical machine learning pipeline

In [53]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# Split the data
X_train_oh, X_test_oh, y_train_oh, y_test_oh = train_test_split(
    X_resampled, y_resampled_one_hot, test_size=0.2, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

# Define candidate models and their hyperparameters
param_grid = [
    {
        'model': [LogisticRegression()],
        'model__penalty': ['l2'],
        'model__C': [0.1, 1.0, 10.0],
        'model__solver': ['lbfgs'],
    },
    # {
    #     'model': [SVC()],
    #     'model__kernel': ['linear', 'rbf'],
    #     'model__C': [0.1, 1.0, 10.0],
    # },
]

param_grid_oh = [
    {
        'model': [RandomForestClassifier()],
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
    },
    {
        'model': [XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', tree_method='gpu_hist')],
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [3, 6],
    },
]

In [7]:
# Define the pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('model', LogisticRegression())  # Placeholder, will be replaced in GridSearchCV
])

# Perform grid search
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', verbose=3)
grid.fit(X_train, y_train)

# Evaluate on test set
y_pred = grid.best_estimator_.predict(X_test)
print("Best Model:", grid.best_estimator_)
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__solver=lbfgs;, score=0.469 total time=   5.3s
[CV 2/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__solver=lbfgs;, score=0.487 total time=   4.7s
[CV 3/3] END model=LogisticRegression(), model__C=0.1, model__penalty=l2, model__solver=lbfgs;, score=0.473 total time=   4.9s
[CV 1/3] END model=LogisticRegression(), model__C=1.0, model__penalty=l2, model__solver=lbfgs;, score=0.701 total time=  24.2s
[CV 2/3] END model=LogisticRegression(), model__C=1.0, model__penalty=l2, model__solver=lbfgs;, score=0.710 total time=  19.0s
[CV 3/3] END model=LogisticRegression(), model__C=1.0, model__penalty=l2, model__solver=lbfgs;, score=0.713 total time=  19.9s


/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/3] END model=LogisticRegression(), model__C=10.0, model__penalty=l2, model__solver=lbfgs;, score=0.801 total time=  28.6s


/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3] END model=LogisticRegression(), model__C=10.0, model__penalty=l2, model__solver=lbfgs;, score=0.801 total time=  27.5s


/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3] END model=LogisticRegression(), model__C=10.0, model__penalty=l2, model__solver=lbfgs;, score=0.804 total time=  28.3s
Best Model: Pipeline(steps=[('model', LogisticRegression(C=10.0))])
              precision    recall  f1-score   support

        enfj       0.80      0.90      0.84       256
        enfp       0.77      0.70      0.73       279
        entj       0.91      0.96      0.93       248
        entp       0.78      0.85      0.81       236
        esfj       0.99      1.00      0.99       251
        esfp       0.95      0.98      0.96       245
        estj       0.99      0.99      0.99       269
        estp       1.00      1.00      1.00       263
        infj       0.60      0.56      0.58       259
        infp       0.55      0.48      0.51       248
        intj       0.75      0.71      0.73       259
        intp       0.76      0.61      0.68       274
        isfj       0.88      0.90      0.89       252
        isfp       0.85      0.93      0.89    

/home/guilherme/Documents/mestrado/disciplinas/inteligencia-artificial/projeto-final/ia-2024-2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
from joblib import dump, load

dump(grid.best_estimator_, 'best_model_regression.joblib')

['best_model_regression.joblib']

In [23]:
clf = load('best_model_regression.joblib')
clf.get_params()

{'memory': None,
 'steps': [('model', LogisticRegression(C=10.0))],
 'transform_input': None,
 'verbose': False,
 'model': LogisticRegression(C=10.0),
 'model__C': 10.0,
 'model__class_weight': None,
 'model__dual': False,
 'model__fit_intercept': True,
 'model__intercept_scaling': 1,
 'model__l1_ratio': None,
 'model__max_iter': 100,
 'model__multi_class': 'deprecated',
 'model__n_jobs': None,
 'model__penalty': 'l2',
 'model__random_state': None,
 'model__solver': 'lbfgs',
 'model__tol': 0.0001,
 'model__verbose': 0,
 'model__warm_start': False}

In [ ]:
# Define the pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('model', LogisticRegression())  # Placeholder, will be replaced in GridSearchCV
])

# Perform grid search
grid = GridSearchCV(pipeline, param_grid_oh, cv=3, scoring='accuracy', verbose=3)
grid.fit(X_train, y_train)

# Evaluate on test set
y_pred = grid.best_estimator_.predict(X_test)
print("Best Model:", grid.best_estimator_)
print(classification_report(y_test, y_pred))


In [24]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

2024-12-10 19:10:23.563537: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 19:10:23.567009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-10 19:10:23.574975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733868623.587883   11794 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733868623.591624   11794 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 19:10:23.607943: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Num GPUs Available: 0


W0000 00:00:1733868624.978339   11794 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [36]:
from sklearn.model_selection import train_test_split

# Split data
X = np.stack(df['glove_embedding'].values)  # Ensure embeddings are stacked correctly
y = np.stack(df['label_one_hot'].values)   # Stack one-hot encoded labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

num_classes = y_train.shape[1]  # Number of classes from one-hot encoding

model = Sequential([
    Dense(128, activation='relu', input_shape=(300,)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Multi-class output
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [50]:
history = model.fit(
	X_train, y_train,
	epochs=100,  # Adjust based on convergence
	batch_size=32,
	validation_data=(X_test, y_test)
)


Epoch 1/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1788 - loss: 2.5156 - val_accuracy: 0.1875 - val_loss: 2.5112
Epoch 2/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1874 - loss: 2.4841 - val_accuracy: 0.1855 - val_loss: 2.5096
Epoch 3/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1785 - loss: 2.4960 - val_accuracy: 0.1868 - val_loss: 2.5100
Epoch 4/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1795 - loss: 2.5042 - val_accuracy: 0.1868 - val_loss: 2.5031
Epoch 5/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1768 - loss: 2.4971 - val_accuracy: 0.1855 - val_loss: 2.5036
Epoch 6/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1803 - loss: 2.4840 - val_accuracy: 0.1836 - val_loss: 2.5133
Epoch 7/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1830 - loss: 2.4974 - val_accuracy: 0.1855 - val_loss: 2.5037
Epoch 8/100
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1787 - loss: 2.4922 - val_accu

In [41]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1813 - loss: 2.7449
Test Accuracy: 18.62%


In [51]:
clf_rf = load('best_model_rf.joblib')
clf_rf.get_params()

{'memory': None,
 'steps': [('model', RandomForestClassifier(n_estimators=200))],
 'transform_input': None,
 'verbose': False,
 'model': RandomForestClassifier(n_estimators=200),
 'model__bootstrap': True,
 'model__ccp_alpha': 0.0,
 'model__class_weight': None,
 'model__criterion': 'gini',
 'model__max_depth': None,
 'model__max_features': 'sqrt',
 'model__max_leaf_nodes': None,
 'model__max_samples': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__monotonic_cst': None,
 'model__n_estimators': 200,
 'model__n_jobs': None,
 'model__oob_score': False,
 'model__random_state': None,
 'model__verbose': 0,
 'model__warm_start': False}

In [56]:
y_pred = clf_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        enfj       0.95      0.89      0.92       256
        enfp       0.92      0.73      0.82       279
        entj       0.99      0.96      0.98       248
        entp       0.94      0.87      0.90       236
        esfj       1.00      0.99      1.00       251
        esfp       1.00      0.97      0.98       245
        estj       1.00      0.99      1.00       269
        estp       1.00      0.99      1.00       263
        infj       0.67      0.69      0.68       259
        infp       0.43      0.79      0.56       248
        intj       0.86      0.81      0.83       259
        intp       0.90      0.68      0.78       274
        isfj       0.96      0.94      0.95       252
        isfp       0.98      0.96      0.97       241
        istj       1.00      0.97      0.98       246
        istp       0.99      0.96      0.97       277

    accuracy                           0.89      4103
   macro avg       0.91   

In [ ]:
# Code to Clean the tweets and make them feasible
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import emoji
import contractions
import spacy
import re

# Load the spaCy English language model
nlp = spacy.load('en_core_web_sm')

def decode_unicode_escape(text):
    return re.sub(r'\\x([0-9a-fA-F]{2})', lambda m: bytes.fromhex(m.group(1)).decode('utf-8', errors='ignore'), text)

def clean_text(text):
    # Decode Unicode escape sequences
    text = decode_unicode_escape(text)

    # Remove Unicode emojis
    text = emoji.emojize(text, language='alias')

    # Case Standardization
    text = text.lower()

    # Remove the leading "b'" and trailing "'"
    text = re.sub(r"^b'|'$", '', text)

    # Remove hyperlinks
    text = re.sub(r'http\S*', '', text)

    # Remove mentions
    text = re.sub(r'@\S*', '', text)

    # Replace \n and \n\n with spaces
    text = re.sub(r'\\n|\\n\\n', ' ', text)

    # Remove any non-alphabetic characters from the text
    text = re.sub(r'[^a-zA-Z\s]+', '', text)

    # Remove hashtags
    text = re.sub(r'#\S*', '', text)

    # Expand contractions
    def expand_contractions(text):
        expanded_words = []
        for word in text.split():
            expanded_words.append(contractions.fix(word))
        return ' '.join(expanded_words)

    text = expand_contractions(text)

    # Lemmatize the text
    def lemmatize(text, nlp):
        doc = nlp(text)
        lemmatized_text = []
        for token in doc:
            lemmatized_text.append(token.lemma_)
        return " ".join(lemmatized_text)

    text = lemmatize(text, nlp)

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]

    # Remove leading '@' tokens that are not valid English words
    # cleaned_tokens = [token for token in tokens if not token.startswith('@')]

    return tokens

tweets_elon_df['clean_content'] = tweets_elon_df['tweet'].apply(clean_text)
print(tweets_elon_df['clean_content'].head())